In [1]:
with open("vis_images.txt", 'r') as f:
    data = f.read().splitlines()

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import FeatureExtract as F
from multiprocessing import Process, Pool
from random import sample
from tqdm import tqdm
from pyDBow3 import Vocabulary
method = "SIFT"
n = 2000
matcher = F.getmatcher(method)

In [3]:
n_imgs= min(20000, len(data))
data = sample(data, n_imgs)
print(len(data),type(data))
n_worker = 8
seg_len = len(data) // n_worker
data_work = [data[i: i + seg_len] for i in range(0, len(data), seg_len)]

20000 <class 'list'>


In [4]:
def process(_data, id):
    print(len(_data))
    desps = []
    for imgpath in tqdm(_data):
        imgname = imgpath.split("/")[-1].split(".")[0]
        img = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(imgname)
        kp, desp = F.extract(img, n, 3, method)
        desps.append(desp)
    desps_all = np.vstack(desps)
    print("worker {} done ".format(id))
    np.save("all_features{}_{}_{}.npy".format(method,id, n_imgs), desps_all) 

In [5]:
with Pool(8) as pool:
    list(tqdm(iterable=(pool.starmap(process, zip(data_work, range(len(data_work))))), total=len(data_work)))

2500


  0%|          | 0/2500 [00:00<?, ?it/s]

2500
2500


  0%|          | 0/2500 [00:00<?, ?it/s]

2500
2500


...

2500


100%|█████████▉| 2493/2500 [29:02<00:12,  1.75s/it]

worker 1 done 


100%|██████████| 2500/2500 [29:11<00:00,  1.43it/s]


worker 3 done 
worker 5 done 
worker 4 done 
worker 6 done 
worker 2 done 


In [4]:
# load all saved npy 
import glob 
desp_files = glob.glob("all_features{}_*_{}.npy".format(method,n_imgs))
desps = []
for file in desp_files:
    desps.append(np.load(file))
    break
desps = np.vstack(desps)

In [ ]:
Level = 4
for Level in [4, 3]:
    voc = Vocabulary(K=10, L=Level,weight_method= "TF_IDF", score_method = "L2_NORM")
    voc.create(desps)
    voc.saveToFile("{}_visdrone_{}_{}.voc".format(method,Level, n_imgs))